In [1]:
# Select you GPU
I_GPU = 0

In [ ]:
# Uncomment to use autoreload
# %load_ext autoreload
# %autoreload 2

import os
import sys
import numpy as np
import torch
import glob
from omegaconf import OmegaConf
import warnings
from kitti360scripts.helpers.ply import read_ply, write_ply
from torch_points3d.metrics.confusion_matrix import ConfusionMatrix
import open3d as o3d
import time
warnings.filterwarnings('ignore')

torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_points3d.trainer import Trainer
from torch_points3d.datasets.segmentation.kitti360_config import KITTI360_NUM_CLASSES
from kitti360scripts.helpers.ply import read_ply, write_ply

In [ ]:
# Set your parameters
DATA_ROOT = '/home/jupyter/datasphere/project/KITTI-360'
checkpoint_dir = '/home/jupyter/datasphere/project'
result_dir = '/home/jupyter/datasphere/project/metrics'
model_name = 'Res16UNet34-PointPyramid-early-cityscapes-interpolate'  # adapt if you use another model in your checkpoint
split = 'val'                                                         # 'test' set will produce data for submission to the KITTI-360 3D semantic segmentation benchmark
n_votes = 1                                                           # number of inferences per cylindrical sample. For multi-inference voting with inference-time augmentation
sample_res = 1                                                        # saptial resolution of inference cylinder samples. Set to 3m for slower inference with +0.2 mIoU, roughly
batch_size = 8                                                        # increase if your device allows it
full_res = True                                                       # predictions will be made on the raw point cloud, at full resolution
num_workers = 4                                                       # increase if your machine allows it
exp_name = None                                                       # you may give a name to the experiment
exp_name = f'{model_name}_{split}_votes-{n_votes}_sample_res-{sample_res}' if exp_name is None else exp_name

In [ ]:
print(f'Inference on KITTI360 exp={exp_name}')

# These are the arguments passed to Hydra. You could run the same thing 
# from CLI using `eval.py` with the command `python eval.py [...]`
overrides = [
    f'model_name={model_name}',
    f'checkpoint_dir={checkpoint_dir}',
    f'voting_runs={n_votes}',
    f'tracker_options.full_res={full_res}',
    f'tracker_options.make_submission={split == "test"}',
    'precompute_multi_scale=False',
    f'num_workers={num_workers}',
    f'batch_size={batch_size}',
    f'cuda={I_GPU}',
    'weight_name=latest',
    f'+data.eval_sample_res={sample_res}',
    f'+data.dataroot={DATA_ROOT}',
    f'+data.mini={True}',
]

# Parse the arguments with Hydra and OmegaConf
cfg = hydra_read(overrides, config_name='eval')
OmegaConf.set_struct(cfg, False)

In [ ]:
def downsample_ply(ply_path, out_path, voxel_size=0.1):
    ply = read_ply(ply_path)
    points = np.array((ply['x'], ply['y'], ply['z'])).T
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    points_down = np.asarray(pcd_down.points)
    # select all the points from ply that are in points_down
    mask = np.isin(points, points_down).all(axis=1)
    # ply_down = {k: ply[k][mask] for k in ply.dtype.names}
    write_ply(out_path, [ply[k][mask] for k in ply.dtype.names], [k for k in ply.dtype.names])

In [ ]:
ply_path = '/home/jupyter/datasphere/project/KITTI-360/kitti360mm/raw/data_3d_semantics/2013_05_28_drive_0000_sync/static/0000000372_0000000610.ply'
ply_path2 = '/home/jupyter/datasphere/project/KITTI-360/kitti360mm/raw/data_3d_semantics/2013_05_28_drive_0000_sync/static/0000000002_0000000385.ply'
downsample_ply(ply_path, ply_path, voxel_size=0.2)
downsample_ply(ply_path2, ply_path2, voxel_size=0.2)
!rm -rf /home/jupyter/datasphere/project/KITTI-360/kitti360mm/processed

In [ ]:
def dva_inference(cfg):
    # Create the Trainer instance from your checkpoint
    trainer = Trainer(cfg)
        
    # Update the val and test transforms to match train transforms for 
    # inference-time augmentation
    trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
    if trainer._model.is_multimodal:
        trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
        trainer._dataset.test_dataset[0].transform_image.transforms[3].use_coverage = False
        trainer._dataset.test_dataset[0].transform_image.transforms[3].credit = int(1408 * 376 * 4 * 2)
        
        # Run inference
    trainer.eval(stage_name=split)
    cm = trainer._tracker._full_confusion_matrix
    
    if split != 'test':
        print(f'  mIoU={cm.get_average_intersection_union() * 100:0.2f}')
        print(f'  OA={cm.get_overall_accuracy() * 100:0.2f}')
    return cm.get_average_intersection_union() * 100

In [ ]:
from unimodal_hdmapnet import eval_semantic

def my_inference():
    ground_truth = read_ply(ply_path2)['semantic']
    semantic = eval_semantic()
    matrix = ConfusionMatrix(KITTI360_NUM_CLASSES)
    return matrix.count_predicted_batch(ground_truth, semantic)